# Apply ARN Categories

This notebook will not run properly in this repository, and is here only for ease-of-access when categorizing chemicals.
The following markdown refers to the [ARN Category repository](https://github.com/pkaramertzanis/regulatory_grouping.git).


This is an adaptation of the code used by the original researchers, based on [model_use_REACH.py](model_use_REACH.py). By pickling the best_rf_model from the original app.py, this code should make the best model from the study usable. 

While it will still take significant time to assign catgories to large inventories, the pickled model is already built, and should not need to be refit or rebuilt each time. This notebook could be used to process any set of chemicals within the applicability domain. The only required information is the CASRN* and SMILES. 

Currently, the code relies on many of the python scripts and file structures contained in the REGULATORY_GROUPING repository, so is best run by cloning the repository and copying in this file and the file best_model_rf.pickle (currently saved to genra-hybrid in Microsoft Teams), after which all processes can be run directly in this notebook. In addition, if cloning the repository from GitHub, you will need to add the folders [output/iteration13/](output/iteration13/) and [logs/](logs/), which are referred to by this code, and place your input file in the [input/](input/) folder. 

*The original code uses CASRN as an indexer. However, this data point is not used otherwise. Slight tweaks to the code should allow you to replace this with DTXSID, if desirable. 

In [2]:
# setup logging
import logger
log = logger.get_logger(__name__)

import matplotlib
matplotlib.use('Tkagg')
# %matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from cheminfo_toolkit import Molecule, Fingerprint_engine
from build_model import group_predictor_rf, group_predictor_kn
import pickle
from pathlib import Path
import numpy as np
import pandas as pd
from model_domain import Domain
from glob import glob
import textwrap

from build_model import build_random_forest_classifier, group_predictor_rf, select_groups

from rdkit import Chem

from visualise_ARN_groups import visualise_ARN_groups
from tqdm import tqdm


The following code prepares the model from the pickle file best_model_rf.pickle, but you will need to adjust the paths to fit your repository structure. When I originally used it, I placed the pickle file in the output/iteration13 folder that is called for by some of the underlying code in this repository. That folder is also where this code will send outputs if not adjusted.

In [10]:
output_folder = Path('output') / 'iteration13'

#Read in the best version of the rf model
with open(output_folder/'best_model_rf.pickle', 'rb') as handle:
    best_model_rf = pickle.load(handle)

# load the molecules (whole dataset)
with open(output_folder/'molecules_all.pickle', 'rb') as handle:
    molecules = pickle.load(handle)
molecules_regrouped = select_groups(molecules,
                                    minimum_group_size=10,
                                    small_groups_as_negative=True,
                                    pulled_small_group_name="miscellaneous chemistry")

#Set the applicability domain
fingerprint_engine = Fingerprint_engine.Morgan(radius=2, nBits=2560)

domain_rf = Domain(molecules_regrouped, fingerprint_engine=fingerprint_engine)

In [11]:
# read the arn groups as produced in app.py
arn_groups = pd.read_excel(output_folder / 'ARN_groups.xlsx')

In [12]:
model_details = best_model_rf

The variable fname below reads in the chemicals of interest. Changing this should allow you to use other chemical inventories for group assignment using this model. 

You may need to change the attribute labels, either in your input file or the following code, for "smiles" and "CASRN".

In [19]:
#Read in the CSV of TSCA chemicals
fname = r'input/tsca_categorisation_071124_wmappingdict.xlsx'
reg_data = pd.read_excel(fname).set_index('Unnamed: 0', drop = True)
reg_cas_numbers = set(reg_data['CASRN'].drop_duplicates())
mol_entries = []
mol_count = 0
mols = reg_data.iloc[12212:].iterrows()
# Iterate over the molecules in the file
for _, mol in tqdm(mols):
    mol_entry = dict(mol)
    try:
        mol = Molecule(Chem.MolFromSmiles(mol['smiles']))
    except:
        continue
    mol_entry['mol'] = mol
    # predicted_groups = group_predictor_rf(mol, best_model_rf['models details']) # <--
    predicted_groups = (
        pd.Series(group_predictor_rf(mol, model_details=best_model_rf['models details'], all_groups=True))
        .sort_values(ascending=False)
        .head(3)
        .rename('group probability')
        .reset_index()
        .rename({'index': 'group name'}, axis='columns'))
    mol_entry['predicted group 1'], mol_entry['predicted group 2'], mol_entry['predicted group 3'] = predicted_groups['group name'].to_list()
    mol_entry['predicted group 1 probability'], mol_entry['predicted group 2 probability'], mol_entry['predicted group 3 probability'] = predicted_groups['group probability'].to_list()
    # mol_entry['predicted group'] = predicted_group
    # mol_entry['probability'] = best_model_rf['models details']['best estimator'].predict_proba(np.array(mol.fingerprint).reshape(1,-1)).max()
    mol_entry['in domain'] = domain_rf.in_domain(mol)
    log.info(f"mol {mol_count} (CAS {mol_entry['CASRN']})  is predicted to belong to group {mol_entry['predicted group 1']} with probability {mol_entry['predicted group 1 probability']: .3f}")
    mol_entries.append(mol_entry)
#             if mol_count > 100:
#                 break
# break
mol_entries = pd.DataFrame(mol_entries)
mol_entries.drop('mol', axis='columns').to_csv(output_folder/'rf_application_1_predicted_groups_only_b.csv')



If working with a new data set, it may be desirable to change the names of the following output files. 

In [26]:
#Join to the main data 
output_data = reg_data.merge(mol_entries, on = list(reg_data.columns.intersection(mol_entries.columns)))
output_data.to_csv('output/iteration13/rf_output_TSCA2.csv')

In [57]:
output_data.head()

,dtxsid,PREFERRED_NAME,CASRN,smiles,errors,qsar_ready_smiles,physical_form,NCC,ClassyFire,group,...,vendor_count,in_sigma_aldrich,mol,predicted group 1,predicted group 2,predicted group 3,predicted group 1 probability,predicted group 2 probability,predicted group 3 probability,in domain
0,DTXSID4063036,1-Nonyne,3452-09-3,CCCCCCCC#C,NaN,CCCCCCCC#C,liquid,"('Neutral Organics',)",Acetylides,"('Acetylides', nan)",...,47.0,1.0,<cheminfo_toolkit.Molecule object at 0x0000025...,Aliphatic nitriles,Simple Lithium compounds,tetrahydroxymethyl and tetraalkyl phosphonium ...,0.462056,0.143764,0.036833,True
1,DTXSID30870753,1-Hexyne,693-02-7,CCCCC#C,NaN,CCCCC#C,liquid,"('Neutral Organics',)",Acetylides,"('Acetylides', nan)",...,63.0,1.0,<cheminfo_toolkit.Molecule object at 0x0000025...,Aliphatic nitriles,Simple Lithium compounds,tetrahydroxymethyl and tetraalkyl phosphonium ...,0.391984,0.137514,0.055325,True
2,DTXSID7062374,"1,8-Nonadiyne",2396-65-8,C#CCCCCCC#C,NaN,C#CCCCCCC#C,liquid,"('Neutral Organics',)",Acetylides,"('Acetylides', nan)",...,39.0,1.0,<cheminfo_toolkit.Molecule object at 0x0000025...,Aliphatic nitriles,primary aliphatic diamines and their salts,Simple Lithium compounds,0.405481,0.170556,0.071101,True
3,DTXSID9061097,1-Pentadecyne,765-13-9,CCCCCCCCCCCCCC#C,NaN,CCCCCCCCCCCCCC#C,liquid,"('Neutral Organics',)",Acetylides,"('Acetylides', nan)",...,46.0,1.0,<cheminfo_toolkit.Molecule object at 0x0000025...,Aliphatic nitriles,Simple Lithium compounds,tetrahydroxymethyl and tetraalkyl phosphonium ...,0.462056,0.143764,0.036833,True
4,DTXSID1061233,"1,7-Octadiyne",871-84-1,C#CCCCCC#C,NaN,C#CCCCCC#C,liquid,"('Neutral Organics',)",Acetylides,"('Acetylides', nan)",...,58.0,1.0,<cheminfo_toolkit.Molecule object at 0x0000025...,Aliphatic nitriles,primary aliphatic diamines and their salts,Simple Lithium compounds,0.410799,0.133905,0.066176,True


In [61]:
output_data.loc[output_data['in domain']==True].to_csv('output/iteration13/rf_output_TSCA_indomainonly2.csv')